https://mp.weixin.qq.com/s/0Y6OYp-bGQf7rFMNnEgSdA

In [1]:
# 导入库
import pandas as pd
import numpy as np
from tqdm import tqdm
# 用于读取和显示图像
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
# 用于创建验证集
from sklearn.model_selection import train_test_split
# 用于评估模型
from sklearn.metrics import accuracy_score
# PyTorch库和模块
import torch
from torch.autograd import Variable
from torch.nn import Linear,ReLU,CrossEntropyLoss,Sequential,Conv2d,MaxPool2d,Module,Softmax,BatchNorm2d,Dropout
from torch.optim import Adam,SGD
# 预训练模型
from torchvision import models
import torch.nn.functional as F

In [2]:
# 加载数据集
train = pd.read_csv('../dataset/emergency_vs_non-emergency_dataset/emergency_train.csv')

In [3]:
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [4]:
# 加载训练图片
train_img = []
for img_name in tqdm(train['image_names']):
    # 定义图像路径
    image_path = '../dataset/emergency_vs_non-emergency_dataset/images/' + img_name
    # 读取图片
    img = imread(image_path)
    # 标准化像素值
    img = img / 255
    img = resize(img,output_shape=(224,224,3),mode='constant',anti_aliasing=True)
    # 转换为浮点数
    img = img.astype('float32')
    # 添加图片到列表
    train_img.append(img)
# 转换为numpy数组
train_x = np.array(train_img)
train_x.shape

100%|█████████████████████████████████████████████████████████████████████████████| 1646/1646 [00:13<00:00, 125.30it/s]


(1646, 224, 224, 3)

In [5]:
# 创建训练和验证集
train_y = train['emergency_or_not'].values # 定义目标
# 创建验证集
X_train,X_test,y_train,y_test = train_test_split(train_x,train_y,test_size=0.1,random_state=13,stratify=train_y)
(X_train.shape,y_train.shape),(X_test.shape,y_test.shape)

(((1481, 224, 224, 3), (1481,)), ((165, 224, 224, 3), (165,)))

In [6]:
# 将图像转换为torch格式
X_train = X_train.reshape(1481,3,224,224)
X_train = torch.from_numpy(X_train)
# 转换目标到torch格式
y_train = y_train.astype(int)
y_train = torch.from_numpy(y_train)
# 转换验证图像为torch格式
X_test = X_test.reshape(165,3,224,224)
X_test = torch.from_numpy(X_test)
# 转换目标到torch格式
y_test = y_test.astype(int)
y_test = torch.from_numpy(y_test)

In [7]:
# 定义模型架构
torch.manual_seed(0)

class Net(Module):
    def __init__(self):
        super(Net,self).__init__()
        self.cnn_layers = Sequential(
            # 定义2D卷积层
            Conv2d(3,16,kernel_size=3,stride=1,padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2,stride=2),
            # 另一层2D卷积层
            Conv2d(16,32,kernel_size=3,stride=1,padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = Sequential(
            Linear(32*56*56,2)
        )
    # 前向传播
    def forward(self,x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0),-1)
        x = self.linear_layers(x)
        return x

In [8]:
# 定义模型参数
model = Net() # 定义模型
# 定义优化器
optimizer = Adam(model.parameters(),lr=0.0001)
# 定义损失函数
criterion = CrossEntropyLoss()
# 检查GPU是否可用
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
  )
)


In [ ]:
# 训练模型
torch.manual_seed(0)
# 模型batch大小
batch_size = 128
# epoch数
n_epochs = 25
for epoch in range(1,n_epochs+1):
    # 保持记录训练与验证集损失
    train_loss = 0.0
    # 打乱数据
    permutation = torch.randperm(X_train.size()[0])
    training_loss = []
    for i in tqdm(range(0,X_train.size()[0],batch_size)):
        indices = permutation[i:i+batch_size]
        batch_x,batch_y = X_train[indices],y_train[indices]
        if torch.cuda.is_available():
            batch_x,batch_y = batch_x.cuda(),batch_y.cuda()
        optimizer.zero_grad()
        
        outputs = model(batch_x)
        print(outputs.dtype)
        print(batch_y.dtype)
        batch_y = torch.tensor(batch_y,dtype=torch.long).cuda()
        loss = criterion(outputs,batch_y)
        
        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

torch.float32
torch.int32


C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
